<a href="https://colab.research.google.com/github/DivyaMeenaSundaram/Agentic-AI/blob/main/Agentic_AI_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sentiment Analysis

In [1]:
!pip install -q langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 862.5 kB/s eta 0:00:00


In [3]:
from google.colab import userdata
import os
import time
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import TransformChain

try:
   GOOGLE_API_KEY = userdata.get('Divya_Key')
   os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
   llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", temperature=0.7) # Slightly higher temp for creativity
   print("API Key loaded and LLM initialized successfully.")
   setup_ok = True
except Exception as e:
   print(f"ERROR: Could not load API key or initialize LLM. {e}")
   print("Please ensure GOOGLE_API_KEY is set in Colab Secrets.")
   setup_ok = False

API Key loaded and LLM initialized successfully.


In [5]:
chain1_template_text = """
You are an expert product reviewer.
Given the product name: "{topic}", generate a comprehensive review of the product.
The review can be only positive or negative or neutral. give more positive or negative reviews than neutral ones. Do not explicitly mention if it is positive, negative or neutral.
"""
chain1_prompt = ChatPromptTemplate.from_template(chain1_template_text)
chain1 = LLMChain(llm=llm, prompt=chain1_prompt)
print("Chain 1 created.")


chain2_template_text = """
Given a product review "{blog_post_idea}",
perform sentiment analysis of the review. Categorize it as positive, negative, or neutral.
Give a list of top 5 short reasons why the review is so.
"""
chain2_prompt = ChatPromptTemplate.from_template(chain2_template_text)
chain2 = LLMChain(llm=llm, prompt=chain2_prompt)
print("Chain 2 created.")


def delay_function(inputs):
      """This function pauses execution and then passes the input through."""
      print("\n--- Pausing for 5s to avoid rate limiting... ---")
      time.sleep(5)
      return inputs


delay_chain = TransformChain(
      input_variables=["blog_post_idea"],
      output_variables=["blog_post_idea"],
      transform=delay_function
)
print("Delay Chain created.")


overall_chain = SimpleSequentialChain(
  chains=[chain1, delay_chain, chain2],
  verbose=True
)


print("Overall SimpleSequentialChain created with a chain 1, delay, chain2.")

Chain 1 created.
Chain 2 created.
Delay Chain created.
Overall SimpleSequentialChain created with a chain 1, delay, chain2.


In [6]:
# --- Run the Overall Chain ---
input_topic = "dark roast coffee powder"
final_response = overall_chain.invoke(input_topic)
print(final_response)



> Entering new SimpleSequentialChain chain...
Here's a review of "Dark Roast Coffee Powder":

The aroma that bursts forth upon opening the bag is intoxicating. A rich, smoky scent, hinting at the depth of flavor to come, immediately fills the air. The powder itself is finely ground, a testament to quality processing, and dissolves readily in hot water.

Brewing this coffee is a simple pleasure. The resulting cup is a deep, inky black, promising a bold experience. The initial sip confirms this promise. A robust, full-bodied flavor profile dominates, with notes of dark chocolate and a hint of caramel sweetness. The bitterness is present, as expected with a dark roast, but it's well-balanced and doesn't overwhelm the palate. The coffee delivers a satisfying, lingering finish that leaves a pleasant warmth.

The caffeine kick is noticeable and provides a welcome boost, perfect for starting the day or powering through an afternoon slump. It performs admirably across various brewing methods